### Imports

In [ ]:
from utils import merge_and_plot, single_file_plot
import os

In [ ]:
SAVE_IMAGE = False

# Save image as file
def save_scatter_image(fig, filename):
    if not os.path.exists("images/scatter"):
        os.mkdir("images/scatter")

    fig.write_image("images/scatter/" + filename + ".pdf")

### 0. Makespan calculation

In [ ]:
folders = ["../solutions/fifo", "../solutions/first_complete"]

fig = merge_and_plot(
    folders, 
    ["FIFO", "First-completion"], 
    "Makespan procedure", 
    "RPD (%)"
)

fig.update_layout(
    xaxis_range=[-0.3,1.3],
    xaxis_title=None
)

fig.show()

if SAVE_IMAGE:
    save_scatter_image(fig, "0_makespan")

### 1. Generational scheme

In [ ]:
folders = ["../solutions/generational", "../solutions/steady_state"]

fig = merge_and_plot(
    folders, 
    ["Generational", "Steady"], 
    "Generational Scheme", 
    "RPD (%)"
)

fig.update_layout(
    xaxis_range=[-0.3,1.3],
    xaxis_title=None
)

fig.show()

if SAVE_IMAGE:
    save_scatter_image(fig, "1_generational")

### Initialisation

In [ ]:
new_names = {"0.2": "GCH (0.2)", "0.5": "GCH (0.5)", "1.0": "GCH (1.0)"}

fig = single_file_plot(
    "../solutions/initialisation", 
    ["construction"], 
    "Initialisation", 
    "RPD (%)",
    new_names
)


fig.show()

if SAVE_IMAGE:
    save_scatter_image(fig, "2_initialisation")

### Population size

In [ ]:
fig = single_file_plot(
    "../solutions/popsize", 
    ["pop_size"], 
    "Population Size", 
    "RPD (%)",
)
fig.show()

if SAVE_IMAGE:
    save_scatter_image(fig, "3_popsize")

### Tournament size

In [ ]:
fig = single_file_plot(
    "../solutions/k_tournament", 
    ["k_tournament"], 
    "Tournament Size", 
    "RPD (%)", 
)

fig.show()

if SAVE_IMAGE:
    save_scatter_image(fig, "4_k_tournament")

### Crossover

In [ ]:
fig = single_file_plot(
    "../solutions/xover-q", 
    ["xover_type"], 
    "Crossover", 
    "RPD (%)", 
)

fig.show()

if SAVE_IMAGE:
    save_scatter_image(fig, "5_crossover")

### Mutation

In [ ]:
fig = single_file_plot(
    "../solutions/mutation-random", 
    ["mutation_type"], 
    "Mutation type", 
    "RPD (%)", 
)

fig.show()

if SAVE_IMAGE:
    save_scatter_image(fig, "6_mutation")

### Mutation probabilities

In [ ]:
fig = single_file_plot(
    "../solutions/mutation-prob", 
    ["mutation_prob"], 
    "Pm", 
    "RPD (%)"
)

fig.show()

if SAVE_IMAGE:
    save_scatter_image(fig, "7_mutation-prob")

### Local search

In [ ]:
fig = single_file_plot(
    "../solutions/local_search", 
    ["approx_calc"], 
    "Iterations", 
    "RPD (%)", 
)

fig.show()

if SAVE_IMAGE:
    save_scatter_image(fig, "8_local-search")

### Crowding Deviation Distance

In [ ]:
fig = single_file_plot(
    "../solutions/crowding-deviation", 
    ["crowding_scale", "k_nearest"], 
    "Crowding scale, k_nearest", 
    "RPD (%)", 
)

fig.update_layout(
        width=650
    )


fig.show()

if SAVE_IMAGE:
    save_scatter_image(fig, "9a_crowding-deviation")

### Crowding Exact Match

In [ ]:
fig = single_file_plot(
    "../solutions/crowding-exact", 
    ["crowding_scale", "k_nearest"], 
    "Crowding scale, k_nearest", 
    "RPD (%)", 
)

fig.update_layout(
        width=600
    )


fig.show()

if SAVE_IMAGE:
    save_scatter_image(fig, "9b_crowding-exact")

### Crowding vs No Crowding

In [ ]:
new_names = {"DeviationDistance": "Deviation Distance", "ExactMatch": "Exact Match"}

fig = single_file_plot(
    "../solutions/crowding-all", 
    ["distance_metric"], 
    "Distance metric", 
    "RPD (%)", 
    new_names
)


fig.show()

if SAVE_IMAGE:
    save_scatter_image(fig, "9c_crowding-all")

### Replacement

In [ ]:
fig = single_file_plot(
    "../solutions/replacement", 
    ["non_improving_iterations", "allways_keep"], 
    "Iterations, Keep", 
    "RPD (%)", 
)

fig.update_layout(
        width=600
    )


fig.show()

if SAVE_IMAGE:
    save_scatter_image(fig, "10a_replacement")

In [ ]:
new_names={"No Replacement, 0": "No replacement"}

fig = single_file_plot(
    "../solutions/replacement-type", 
    ["rtype", "non_improving_iterations"], 
    "Replacement scheme, iterations", 
    "RPD (%)", 
    new_names
)

fig.update_layout(
        width=600,
        xaxis_title=None
)


fig.show()

if SAVE_IMAGE:
    save_scatter_image(fig, "10b_replacement-type")

In [ ]:
new_names={"0.0": "No replacement"}

fig = single_file_plot(
    "../solutions/replacement-rate", 
    ["allways_keep"], 
    "Replacement rate", 
    "RPD (%)", 
    new_names
)

fig.update_layout(
        width=600,
)


fig.show()

if SAVE_IMAGE:
    save_scatter_image(fig, "10c_replacement_rate")

### Iterated Greedy

In [ ]:
fig = single_file_plot(
    "../solutions/ig-params", 
    ["t", "d"],
    "T, d",
    "RPD (%)", 
)

fig.update_layout(
        width=600
    )


fig.show()

if SAVE_IMAGE:
    save_scatter_image(fig, "12_iterated-greedy")

### Q-Learning Scaled

In [ ]:
fig = single_file_plot(
    "../solutions/qlearning", 
    ["learning_rate", "epsilon"], 
    "Learning rate, epsilon", 
    "RPD (%)", 
)

fig.show()

if SAVE_IMAGE:
    save_scatter_image(fig, "5b_qlearning")

### Q-Learning Non-Scaled Time

In [ ]:
fig = single_file_plot(
    "../solutions/qlearning-noscale", 
    ["learning_rate", "epsilon"], 
    "Learning rate, epsilon", 
    "RPD (%)", 
)

fig.show()

if SAVE_IMAGE:
    save_scatter_image(fig, "5c_qlearning-noscale")